In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import datetime as dt
import json
import csv
import pytemperature as pyt
from pprint import pprint

# Import API key
from api_keys import api_key

# files to Load
cpd_crime2018_load = "Resources/Crimes_-_2018.csv"
cpd_crime2017_load = "Resources/Crimes_-_2017.csv"
cpd_crime2016_load = "Resources/Crimes_-_2016.csv"
cpd_crime2015_load = "Resources/Crimes_-_2015.csv"
cpd_crime2014_load = "Resources/Crimes_-_2014.csv"

# read CPD crime data and store
cpdcrime2018_data = pd.read_csv(cpd_crime2018_load)
cpdcrime2017_data = pd.read_csv(cpd_crime2017_load)
cpdcrime2016_data = pd.read_csv(cpd_crime2016_load)
cpdcrime2015_data = pd.read_csv(cpd_crime2015_load)
cpdcrime2014_data = pd.read_csv(cpd_crime2014_load)


In [2]:
#pull column data and drop NaN 
cpdcrime2018_data = cpdcrime2018_data[['Date', 'Primary Type']]
cpdcrime2018_data = cpdcrime2018_data.dropna(how='any')

cpdcrime2017_data = cpdcrime2017_data[['Date', 'Primary Type']]
cpdcrime2017_data = cpdcrime2017_data.dropna(how='any')

cpdcrime2016_data = cpdcrime2016_data[['Date', 'Primary Type']]
cpdcrime2016_data = cpdcrime2016_data.dropna(how='any')

cpdcrime2015_data = cpdcrime2015_data[['Date', 'Primary Type']]
cpdcrime2015_data = cpdcrime2015_data.dropna(how='any')

cpdcrime2014_data = cpdcrime2014_data[['Date', 'Primary Type']]
cpdcrime2014_data = cpdcrime2014_data.dropna(how='any')


In [3]:
#use to check outputs
#cpdcrime2018_data.head()
#cpdcrime2017_data.head()
#cpdcrime2016_data.head()
#cpdcrime2015_data.head()
#cpdcrime2014_data.head()

In [4]:
#create dataframe with the date of each reported incident by primary type
battery18 = cpdcrime2018_data[cpdcrime2018_data['Primary Type'] == "BATTERY"]["Date"].reset_index()
battery17 = cpdcrime2017_data[cpdcrime2017_data['Primary Type'] == "BATTERY"]["Date"].reset_index()
battery16 = cpdcrime2016_data[cpdcrime2016_data['Primary Type'] == "BATTERY"]["Date"].reset_index()
battery15 = cpdcrime2015_data[cpdcrime2015_data['Primary Type'] == "BATTERY"]["Date"].reset_index()
battery14 = cpdcrime2014_data[cpdcrime2014_data['Primary Type'] == "BATTERY"]["Date"].reset_index()

#format date in dataframe to be mergeable with weather data 
battery18['Date'] = pd.to_datetime(battery18['Date']).dt.date
battery18['Date'] = pd.to_datetime(battery18['Date'], infer_datetime_format=True)
battery17['Date'] = pd.to_datetime(battery17['Date']).dt.date
battery17['Date'] = pd.to_datetime(battery17['Date'], infer_datetime_format=True)
battery16['Date'] = pd.to_datetime(battery16['Date']).dt.date
battery16['Date'] = pd.to_datetime(battery16['Date'], infer_datetime_format=True)
battery15['Date'] = pd.to_datetime(battery15['Date']).dt.date
battery15['Date'] = pd.to_datetime(battery15['Date'], infer_datetime_format=True)
battery14['Date'] = pd.to_datetime(battery14['Date']).dt.date
battery14['Date'] = pd.to_datetime(battery14['Date'], infer_datetime_format=True)

#delete unnecessary index column
del battery18['index']
del battery17['index']
del battery16['index']
del battery15['index']
del battery14['index']

#merge all reported battery incidents for all five years
battery_all = battery18.append([battery17, battery16, battery15, battery14])

In [5]:
#use to check output
#battery18
#battery18.dtypes
#battery17
#battery17.dtypes
#battery16
#battery16.dtypes
#battery15
#battery15.dtypes
#battery14
#battery14.dtypes
#battery_all
#battery_all.dtypes

In [6]:
#create dataframe with the date of each reported incident by primary type
csa18 = cpdcrime2018_data[cpdcrime2018_data['Primary Type'] == "CRIM SEXUAL ASSAULT"]["Date"].reset_index()
csa17 = cpdcrime2017_data[cpdcrime2017_data['Primary Type'] == "CRIM SEXUAL ASSAULT"]["Date"].reset_index()
csa16 = cpdcrime2016_data[cpdcrime2016_data['Primary Type'] == "CRIM SEXUAL ASSAULT"]["Date"].reset_index()
csa15 = cpdcrime2015_data[cpdcrime2015_data['Primary Type'] == "CRIM SEXUAL ASSAULT"]["Date"].reset_index()
csa14 = cpdcrime2014_data[cpdcrime2014_data['Primary Type'] == "CRIM SEXUAL ASSAULT"]["Date"].reset_index()

#format date in dataframe to be mergeable with weather data 
csa18['Date'] = pd.to_datetime(csa18['Date']).dt.date
csa18['Date'] = pd.to_datetime(csa18['Date'], infer_datetime_format=True)
csa17['Date'] = pd.to_datetime(csa17['Date']).dt.date
csa17['Date'] = pd.to_datetime(csa17['Date'], infer_datetime_format=True)
csa16['Date'] = pd.to_datetime(csa16['Date']).dt.date
csa16['Date'] = pd.to_datetime(csa16['Date'], infer_datetime_format=True)
csa15['Date'] = pd.to_datetime(csa15['Date']).dt.date
csa15['Date'] = pd.to_datetime(csa15['Date'], infer_datetime_format=True)
csa14['Date'] = pd.to_datetime(csa14['Date']).dt.date
csa14['Date'] = pd.to_datetime(csa14['Date'], infer_datetime_format=True)

#delete unnecessary index column
del csa18['index']
del csa17['index']
del csa16['index']
del csa15['index']
del csa14['index']

#merge all reported criminal sexual assault incidents for all five years
csa_all = csa18.append([csa17, csa16, csa15, csa14])

In [ ]:
#use to check output
#csa18
#csa18.dtypes
#csa17
#csa17.dtypes
#csa16
#csa16.dtypes
#csa15
#csa15.dtypes
#csa14
#csa14.dtypes
#csa_all
#csa_all.dtypes

In [ ]:
#create dataframe with the date of each reported incident by primary type
homicide18 = cpdcrime2018_data[cpdcrime2018_data['Primary Type'] == "HOMICIDE"]["Date"].reset_index()
homicide17 = cpdcrime2017_data[cpdcrime2017_data['Primary Type'] == "HOMICIDE"]["Date"].reset_index()
homicide16 = cpdcrime2016_data[cpdcrime2016_data['Primary Type'] == "HOMICIDE"]["Date"].reset_index()
homicide15 = cpdcrime2015_data[cpdcrime2015_data['Primary Type'] == "HOMICIDE"]["Date"].reset_index()
homicide14 = cpdcrime2014_data[cpdcrime2014_data['Primary Type'] == "HOMICIDE"]["Date"].reset_index()

#format date in dataframe to be mergeable with weather data 
homicide18['Date'] = pd.to_datetime(homicide18['Date']).dt.date
homicide18['Date'] = pd.to_datetime(homicide18['Date'], infer_datetime_format=True)
homicide17['Date'] = pd.to_datetime(homicide17['Date']).dt.date
homicide17['Date'] = pd.to_datetime(homicide17['Date'], infer_datetime_format=True)
homicide16['Date'] = pd.to_datetime(homicide16['Date']).dt.date
homicide16['Date'] = pd.to_datetime(homicide16['Date'], infer_datetime_format=True)
homicide15['Date'] = pd.to_datetime(homicide15['Date']).dt.date
homicide15['Date'] = pd.to_datetime(homicide15['Date'], infer_datetime_format=True)
homicide14['Date'] = pd.to_datetime(homicide14['Date']).dt.date
homicide14['Date'] = pd.to_datetime(homicide14['Date'], infer_datetime_format=True)

#delete unnecessary index column
del homicide18['index']
del homicide17['index']
del homicide16['index']
del homicide15['index']
del homicide14['index']


#merge all reported homicide incidents for all five years
homicide_all = homicide18.append([homicide17, homicide16, homicide15, homicide14])

In [ ]:
#use to check output
#homicide18
#homicide18.dtypes
#homicide17
#homicide17.dtypes
#homicide16
#homicide16.dtypes
#homicide15
#homicide15.dtypes
#homicide14
#homicide14.dtypes
#homicide_all
#homicide_all.dtypes

In [ ]:
#create dataframe with the date of each reported incident by primary type
robbery18 = cpdcrime2018_data[cpdcrime2018_data['Primary Type'] == "ROBBERY"]["Date"].reset_index()
robbery17 = cpdcrime2017_data[cpdcrime2017_data['Primary Type'] == "ROBBERY"]["Date"].reset_index()
robbery16 = cpdcrime2016_data[cpdcrime2016_data['Primary Type'] == "ROBBERY"]["Date"].reset_index()
robbery15 = cpdcrime2015_data[cpdcrime2015_data['Primary Type'] == "ROBBERY"]["Date"].reset_index()
robbery14 = cpdcrime2014_data[cpdcrime2014_data['Primary Type'] == "ROBBERY"]["Date"].reset_index()

#format date in dataframe to be mergeable with weather data 
robbery18['Date'] = pd.to_datetime(robbery18['Date']).dt.date
robbery18['Date'] = pd.to_datetime(robbery18['Date'], infer_datetime_format=True)
robbery17['Date'] = pd.to_datetime(robbery17['Date']).dt.date
robbery17['Date'] = pd.to_datetime(robbery17['Date'], infer_datetime_format=True)
robbery16['Date'] = pd.to_datetime(robbery16['Date']).dt.date
robbery16['Date'] = pd.to_datetime(robbery16['Date'], infer_datetime_format=True)
robbery15['Date'] = pd.to_datetime(robbery15['Date']).dt.date
robbery15['Date'] = pd.to_datetime(robbery15['Date'], infer_datetime_format=True)
robbery14['Date'] = pd.to_datetime(robbery14['Date']).dt.date
robbery14['Date'] = pd.to_datetime(robbery14['Date'], infer_datetime_format=True)

#delete unnecessary index column
del robbery18['index']
del robbery17['index']
del robbery16['index']
del robbery15['index']
del robbery14['index']

#merge all reported robbery incidents for all five years
robbery_all = robbery18.append([robbery17, robbery16, robbery15, robbery14])

In [ ]:
#use to check output
#robbery18
#robbery18.dtypes
#robbery17
#robbery17.dtypes
#robbery16
#robbery16.dtypes
#robbery15
#robbery15.dtypes
#robbery14
#robbery14.dtypes
#robbery_all
#robbery_all.dtypes

In [ ]:
Datetime = []
Max_Temp = []
Humidity = []
Cloudiness = []

In [ ]:
file = open("weather_data.json")
json_data = json.load(file)

In [ ]:
for each in json_data["values"]:
    Datetime.append(each["dt"])
    Max_Temp.append(each["main"]["temp_max"])
    Humidity.append(each["main"]["humidity"])
    Cloudiness.append(each["clouds"]["all"])

In [ ]:
weather_df1 = pd.DataFrame({"Datetime": Datetime,
                            "Max_Temp": Max_Temp,
                            "Humidity": Humidity,
                            "Cloudiness": Cloudiness})

weather_df1.head()

In [ ]:
weather_df1["Datetime"]=pd.to_datetime(weather_df1["Datetime"],unit='s')
weather_df1.head(5)

In [ ]:
weather_df1["Date"] = pd.to_datetime(weather_df1['Datetime']).dt.date
weather_df1["Date"] = pd.to_datetime(weather_df1['Date'], infer_datetime_format=True)
weather_df1["Time"] = pd.to_datetime(weather_df1['Datetime']).dt.time
weather_df1["Time"] = pd.to_datetime(weather_df1['Datetime'], infer_datetime_format=True)


weather_df1.head()

In [ ]:
weather_df1["Max_Temp_(F)"]=pyt.k2f(weather_df1["Max_Temp"])
weather_df1.head()

In [ ]:
# del weather_df1['Datetime']
del weather_df1["Max_Temp"]
del weather_df1["Datetime"]
weather_df1.head()

In [ ]:
weather_df2 = pd.DataFrame(weather_df1.groupby("Date")["Humidity", "Cloudiness", "Max_Temp_(F)"].mean())
weather_df2.head()